## 多线程(时间片轮转)

In [7]:
import threading
import time

def dance():
    for i in range(5):
        print("dancing")
        time.sleep(1)

def sing():
    for i in range(5):
        print("singing")
        time.sleep(1)
    
def main():
    dance()
    sing()
    
if __name__ == "__main__":
    main()

dancing
dancing
dancing
dancing
dancing
singing
singing
singing
singing
singing


In [13]:
# 并发随机分配执行
import threading
import time

def dance():
    for i in range(5):
        print("dancing")
        time.sleep(1)

def sing():
    for i in range(5):
        print("singing")
        time.sleep(1)
    
def main():
    t1 = threading.Thread(target=dance, args=())
    t2 = threading.Thread(target=sing, args=())
    t1.start()
    t2.start()
    
if __name__ == "__main__":
    main()

dancing
singing
dancingsinging

singingdancing

dancingsinging

dancing
singing


```
并行：真的多任务
并发：假的多任务
```

In [15]:
a = ["aa", "bb", "cc"]
for num, value in enumerate(a):
    print(num, value)

0 aa
1 bb
2 cc


In [23]:
# 查看当前运行线程
# threading.enumerate()函数查看当前运行线程，返回列表
x = threading.enumerate()
print(x)
print("-"*30)
print("当前运行线程数：%d" % len(x))

[<_MainThread(MainThread, started 4320768832)>, <Thread(Thread-2, started daemon 123145477660672)>, <Heartbeat(Thread-3, started daemon 123145482915840)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145489244160)>, <ParentPollerUnix(Thread-1, started daemon 123145494499328)>]
------------------------------
当前运行线程数：5


### 使用类重写创建线程

In [27]:
import threading
import time

class MyThread(threading.Thread):
    def run(self):
        self.num()
    def num(self):
        for i in range(5):
            time.sleep(1)
            print(len(threading.enumerate()))
            
if __name__ == "__main__":
    t = MyThread()
    t.start()

6
6
6
6
6


### 函数修改全局变量
- 修改引用需要添加global
- 修改引用的数据不需要添加

In [35]:
# 修改全局变量实例
a = 10
b = [1,2]
def cha():
    global a
    a += 10
    print(a)

def chb():
    b.append(3)
    print(b)

print(a)
cha()
print(b)
chb()

10
20
[1, 2]
[1, 2, 3]


In [38]:
# 列表中 += 相当于 extend()
# 正常情况 += 就是 =。。+。。
a = [1]
print(id(a))
a = a + [1]
print(id(a))
a += [1]
print(id(a))

4440680904
4444792712
4444792712


In [39]:
a = b = 1
a += 1
print(a,b)

2 1


### 互斥锁
- 锁只能上锁一次，其他程序上锁会阻塞
- 互斥锁保证线程的程序执行完才会切换
- 尽量把上锁代码量减少

In [40]:
# 创建锁
mutex = threading.Lock()
# 上锁
mutex.acquire()
# 解锁
mutex.release()

In [47]:
import threading
import time

num = 0

def a():
    global num
    for i in range(10000000):
        num += 1
    print("a:%d" % num)
    
def b():
    global num
    for i in range(10000000):
        num += 1
    print("b:%d" % num)
    
def main():
    t1 = threading.Thread(target=a)
    t2 = threading.Thread(target=b)
    t1.start()
    t2.start()
    time.sleep(5)
    print("main:%d" % num)
    
if __name__ == "__main__":
    main()

b:13287703
a:13412191
main:13412191


In [51]:
# 循坏外上锁可以节省时间，但阻塞时间增加
import threading
import time

num = 0
m = threading.Lock()

def a():
    global num
    m.acquire()
    for i in range(10000000):
        num += 1
    m.release()
    print("a:%d" % num)
    
def b():
    global num
    m.acquire()
    for i in range(10000000):
        num += 1
    m.release()
    print("b:%d" % num)
    
def main():
    t1 = threading.Thread(target=a)
    t2 = threading.Thread(target=b)
    t1.start()
    t2.start()
    time.sleep(5)
    print("main:%d" % num)
    
if __name__ == "__main__":
    main()

a:10000000
b:20000000
main:20000000


In [52]:
#循环内上锁的话，上锁解锁次数多运行时间较长，但阻塞时间较短
import threading
import time

num = 0
m = threading.Lock()

def a():
    global num
    for i in range(10000000):
        m.acquire()
        num += 1
        m.release()
    print("a:%d" % num)
    
def b():
    global num
    for i in range(10000000):
        m.acquire()
        num += 1
        m.release()
    print("b:%d" % num)
    
def main():
    t1 = threading.Thread(target=a)
    t2 = threading.Thread(target=b)
    t1.start()
    t2.start()
    time.sleep(5)
    print("main:%d" % num)
    
if __name__ == "__main__":
    main()

main:12747294
a:19344496
b:20000000
